# Chapter 2

# OpenAI API prompt examples

## Minimal prompt execution

In [2]:
from openai import OpenAI
import getpass

OPENAI_API_KEY = getpass.getpass('Enter your OPENAI_API_KEY') 

Enter your OPENAI_API_KEY ········


In [3]:
client = OpenAI(api_key=OPENAI_API_KEY)

In [4]:
prompt_input = """Write a coincise message to remind users to be vigilant about phishing attacks."""
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt_input}
  ],
  temperature= 0.7,
  max_tokens= 400  
)

print(response)

ChatCompletion(id='chatcmpl-AIkMOj6jqpvKds7L6cmuUQXaEY3xw', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Subject: Stay Vigilant Against Phishing Attacks\n\nDear Team,\n\nAs a reminder, please remain vigilant against phishing attacks. Always verify the sender's email address, avoid clicking on suspicious links, and never share your personal information. If something seems off, report it immediately.\n\nStay safe!\n\nBest,  \n[Your Name]  \n[Your Position]", refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1729031476, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_e2bde53e6e', usage=CompletionUsage(completion_tokens=70, prompt_tokens=32, total_tokens=102, prompt_tokens_details={'cached_tokens': 0}, completion_tokens_details={'reasoning_tokens': 0}))


# Running prompts with LangChain

## Basic prompt with LangChain

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY,
                 model_name="gpt-4o-mini")

In [6]:
prompt_input = """Write a coincise message to remind users to be vigilant about phishing attacks."""

response = llm.invoke(prompt_input)
print(response.content)

🔔 Reminder: Stay vigilant against phishing attacks! Always verify the sender's email address, avoid clicking on suspicious links, and never share personal information. Protect yourself and your data! 🛡️


# Prompt templates

## Prompt template - implementing it with a Python function

In [1]:
def generate_text_summary_prompt(text, num_words, tone):
    return f"You are an experienced copywriter. Write a {num_words} words summary the the following text, using a {tone} tone: {text}"

In [9]:
segovia_aqueduct_text = "The Aqueduct of Segovia (Spanish: Acueducto de Segovia) is a Roman aqueduct in Segovia, Spain. It was built around the first century AD to channel water from springs in the mountains 17 kilometres (11 mi) away to the city's fountains, public baths and private houses, and was in use until 1973. Its elevated section, with its complete arcade of 167 arches, is one of the best-preserved Roman aqueduct bridges and the foremost symbol of Segovia, as evidenced by its presence on the city's coat of arms. The Old Town of Segovia and the aqueduct, were declared a UNESCO World Heritage Site in 1985. As the aqueduct lacks a legible inscription (one was apparently located in the structure's attic, or top portion[citation needed]), the date of construction cannot be definitively determined. The general date of the Aqueduct's construction was long a mystery, although it was thought to have been during the 1st century AD, during the reigns of the Emperors Domitian, Nerva, and Trajan. At the end of the 20th century, Géza Alföldy deciphered the text on the dedication plaque by studying the anchors that held the now missing bronze letters in place. He determined that Emperor Domitian (AD 81–96) ordered its construction[1] and the year 98 AD was proposed as the most likely date of completion.[2] However, in 2016 archeological evidence was published which points to a slightly later date, after 112 AD, during the government of Trajan or in the beginning of the government of emperor Hadrian, from 117 AD."

input_prompt = generate_text_summary_prompt(text=segovia_aqueduct_text, num_words=20, tone="knowledgeable and engaging")

response = llm.invoke(input_prompt)
print(response.content)

The Aqueduct of Segovia, a stunning Roman engineering marvel, dates back to 1st century AD, symbolizing the city's rich heritage.


## Prompt template - using LangChain's ChatPromptTemplate

In [10]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("You are an experienced copywriter. Write a {num_words} words summary the the following text, using a {tone} tone: {text}")

In [11]:
prompt = prompt_template.format(text=segovia_aqueduct_text, num_words=20, tone="knowledgeable and engaging")

In [12]:
response = llm.invoke(prompt)
print(response.content)

The Aqueduct of Segovia, a UNESCO World Heritage Site, is a remarkable Roman engineering feat, dating back to the 1st century AD.


# Few shot prompt with LangChain

## Using standard prompt

In [13]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY,
                 model_name="gpt-4o-mini")

In [14]:
prompt_input = """Classify the following numbers as Abra, Kadabra or Abra Kadabra:

3, 4, 5, 7, 8, 10, 11, 13, 35

Examples: 
6 // not divisible by 5, not divisible by 7 // None
15 // divisible by 5, not divisible by 7 // Abra
12 // not divisible by 5, not divisible by 7 // None
21 // not divisible by 5, divisible by 7 // Kadabra
70 // divisible by 5, divisible by 7 // Abra Kadabra
"""

response = llm.invoke(prompt_input)
print(response.content)

Let's classify the given numbers based on their divisibility by 5 and 7:

1. **3**: Not divisible by 5, not divisible by 7 // None
2. **4**: Not divisible by 5, not divisible by 7 // None
3. **5**: Divisible by 5, not divisible by 7 // Abra
4. **7**: Not divisible by 5, divisible by 7 // Kadabra
5. **8**: Not divisible by 5, not divisible by 7 // None
6. **10**: Divisible by 5, not divisible by 7 // Abra
7. **11**: Not divisible by 5, not divisible by 7 // None
8. **13**: Not divisible by 5, not divisible by 7 // None
9. **35**: Divisible by 5, divisible by 7 // Abra Kadabra

Here's the classification:

- **3**: None
- **4**: None
- **5**: Abra
- **7**: Kadabra
- **8**: None
- **10**: Abra
- **11**: None
- **13**: None
- **35**: Abra Kadabra


## Using Langchain's FewShotPromptTemplate

In [25]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

examples = [
  {
      "number": 6,
      "reasoning": "not divisible by 5 nor by 7",
      "result": "None"
  },
  {
      "number": 15,
      "reasoning": "divisible by 5 but not by 7",
      "result": "Abra"
  },
  {
      "number": 12,
      "reasoning": "not divisible by 5 nor by 7",
      "result": "None"
  },
  {
      "number": 21,
      "reasoning": "divisible by 7 but not by 5",
      "result": "Kadabra"
  },
  {
      "number": 70,
      "reasoning": "divisible by 5 and by 7",
      "result": "Abra Kadabra"
  } ]

example_prompt = PromptTemplate(input_variables=["number", "reasoning", "result"], template="{number} \\ {reasoning} \\ {result}")
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Classify the following numbers as Abra, Kadabra or Abra Kadabra: {comma_delimited_input_numbers}",
    input_variables=["comma_delimited_input_numbers"]
)

prompt_input = few_shot_prompt.format(comma_delimited_input_numbers="3, 4, 5, 7, 8, 10, 11, 13, 35.")

response = llm.invoke(prompt_input)
print(response.content)

3 \ Abra
4 \ Abra
5 \ Abra
7 \ Kadabra
8 \ Abra
10 \ Abra
11 \ Abra
13 \ Abra
35 \ Abra Kadabra
